In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import scipy.optimize
import scipy.stats
from scipy.io import loadmat
from datetime import datetime, date, time, timedelta
sns.set()

import ipynb.fs.defs.analysis_fxns as af

# import mat files

In [5]:
path = 'uva_data/dataCollDates.mat'
mat = scipy.io.loadmat(path)

In [6]:
data = [[row.flat[0] for row in line] for line in mat['dataCollDates']]
df_collDates = pd.DataFrame(data)
df_collDates.columns = df_collDates.iloc[0]
df_collDates = df_collDates[1:]
df_collDates = df_collDates.reset_index(drop = True)

In [7]:
path = 'uva_data/subjData.mat'
mat = scipy.io.loadmat(path)

In [8]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data', 'dates', 'demo'])

In [9]:
# formatting mat data into a dataframe
data = [[row for row in line] for line in mat['dates']]
df_dates = pd.DataFrame(data)

df_dates.columns = ['ID', 'Menses dates', 'Ovulation dates', 'Group']
df_dates = df_dates[1:]
df_dates = df_dates.reset_index(drop = True)

In [10]:
# these columns were arrays of arrays ( [[data]] ) - getting rid of the extra shell 
df_dates['ID'] = df_dates['ID'].str[0]
df_dates['Group'] = df_dates['Group'].str[0]

In [11]:
# making columns of strings for mens/ovulation dates from matlab hash
str_mens_dates = []
str_ov_dates = []
for i in range(len(df_dates)):
    menses = df_dates['Menses dates'][i]
    subj_str_mens = []
    
    for j in range(len(menses)):
        date = menses[j][0]
        dt = datetime.fromordinal(int(date)) + timedelta(days=int(date)%1) - timedelta(days = 366)
        subj_str_mens.append(dt.strftime('%d-%b-%Y'))
    str_mens_dates.append(subj_str_mens)
    
    ovs = df_dates['Ovulation dates'][i]
    subj_str_ovs = []
    
    for j in range(len(ovs)):
        date = ovs[j][0]
        dt = datetime.fromordinal(int(date)) + timedelta(days=int(date)%1) - timedelta(days = 366)
        subj_str_ovs.append(dt.strftime('%d-%b-%Y'))
    str_ov_dates.append(subj_str_ovs)

In [12]:
df_dates['Menses dates (str)'] = str_mens_dates
df_dates['Ovulation dates (str)'] = str_ov_dates

In [13]:
df_dates.head()

,ID,Menses dates,Ovulation dates,Group,Menses dates (str),Ovulation dates (str)
0,[78101],"[[738227], [738256], [738281]]","[[738211], [738240], [738273], [738293]]",[0],"[12-Mar-2021, 10-Apr-2021, 05-May-2021]","[24-Feb-2021, 25-Mar-2021, 27-Apr-2021, 17-May..."
1,[78103],"[[738238], [738264], [738293]]","[[738252], [738280], [738306]]",[0],"[23-Mar-2021, 18-Apr-2021, 17-May-2021]","[06-Apr-2021, 04-May-2021, 30-May-2021]"
2,[78104],"[[738250], [738291], [738319]]",[],[1],"[04-Apr-2021, 15-May-2021, 12-Jun-2021]",[]
3,[78105],"[[738201], [738227], [738253]]","[[738211], [738238], [738264]]",[0],"[14-Feb-2021, 12-Mar-2021, 07-Apr-2021]","[24-Feb-2021, 23-Mar-2021, 18-Apr-2021]"
4,[78108],"[[738302], [738327], [738358]]","[[738316], [738342], [738371]]",[0],"[26-May-2021, 20-Jun-2021, 21-Jul-2021]","[09-Jun-2021, 05-Jul-2021, 03-Aug-2021]"


# import not mat files

In [14]:
# get files 
data = glob.glob('uva_data/ind_data/*****_data.csv')

In [15]:
# make files into dataframes
dfs = []
for file in data:
    dfs.append(pd.read_csv(file, low_memory = False))

In [16]:
def add_dates(df, df_dates):
    '''adds bool columns for menses and ovulation dates'''
    # get subject id, use to get menses/ovulation dates 
    subj_id = df['id'][0] 
    mens_dates = list(df_dates[df_dates['ID'] == subj_id]['Menses dates (str)'])[0]
    ov_dates =  list(df_dates[df_dates['ID'] == subj_id]['Ovulation dates (str)'])[0]
    
    # split t into date/time columns
    df[['date', 'time']] = df['t'].str.split(pat = ' ', expand = True)
    
    # makes new columns with bool value if date is in 
    df['is mens'] = np.where(df['date'].isin(mens_dates), 1, 0)
    df['is ov'] = np.where(df['date'].isin(ov_dates), 1, 0)
    
    # makes sure t is still in datetime format
    df['t'] = pd.to_datetime(df['t'])

In [17]:
for df in dfs:
    add_dates(df, df_dates)

In [18]:
dfs[0].head()

,id,t,cgm,basal,bolus,meals,pump,clc,date,time,is mens,is ov
0,78132,2021-08-06 00:00:00,361.0,1.179,0.0,0,NaN,NaN,06-Aug-2021,00:00:00,0,0
1,78132,2021-08-06 00:05:00,355.0,1.530,0.0,0,NaN,NaN,06-Aug-2021,00:05:00,0,0
2,78132,2021-08-06 00:10:00,348.0,1.802,0.0,0,NaN,NaN,06-Aug-2021,00:10:00,0,0
3,78132,2021-08-06 00:15:00,341.0,2.006,0.0,0,NaN,NaN,06-Aug-2021,00:15:00,0,0
4,78132,2021-08-06 00:20:00,340.0,2.158,0.0,0,NaN,NaN,06-Aug-2021,00:20:00,0,0


In [20]:
def days_since_mens(df, df_dates):
    '''adding dates since mens column and phase column'''
    
    # getting mens/ov dates and all dates in list form
    subj_id = df.iloc[0]['id']
    mens_dates_t = list(df_dates[df_dates['ID'] == subj_id]['Menses dates (str)'])[0]
    ov_dates_t = list(df_dates[df_dates['ID'] == subj_id]['Ovulation dates (str)'])[0]
    dates = list(df['date'])

    days_since = np.zeros(len(df))
    phase_list = np.zeros(len(df))
    before_first = True
    next_mens = mens_dates_t[0]
    
    if ov_dates_t != []: # if ovulation dates exist
        next_ov = ov_dates_t[0]
        ov_dates_exist = True
        phase = 1 # 1 for follicular, 2 for luteal
        
    else: # if ovulation dates don't exist, midpoint between menses dates is used instead 
        # FIX: need to implement finding midpoint for ovulation dates 
        ov_dates_exist = False
    mens_count = 0
    last_date = dates[0]
    reset = False

    for i in range(len(dates)):

        # if date matches menses date, reset count and set next_mens to next date, set phase back to follicular
        if dates[i] == next_mens:
            before_first = False
            if mens_count == len(mens_dates_t) - 1:
                next_mens = '01-Jan-2000' # dummy date so it counts correctly 
                days_since_count = 0
                if ov_dates_exist:
                    phase = 1
                reset = True
                continue
            else:
                next_mens = mens_dates_t[mens_count + 1]
            mens_count += 1
            days_since_count = 0
            
        if ov_dates_exist:
            if dates[i] == next_ov: # changes phase if past ovulation but does nothing else
                phase = 2

        #if before the first menses date provided, set pos as nan
        if before_first == True:
            days_since[i] = np.nan
            if ov_dates_exist:
                phase_list[i] = phase
            else:
                phase_list[i] = np.nan

        else:
            #if date is new, count + 1 and set last_date to date 
            if last_date != dates[i]:
                if reset == True:
                    reset = False
                else:
                    days_since_count += 1 
            last_date = dates[i]
            days_since[i] = days_since_count
            if ov_dates_exist:
                phase_list[i] = phase
            else:
                phase_list[i] = np.nan
        
    # add to df        
    df['days since'] = days_since
    df['phase'] = phase_list

In [21]:
for df in dfs:
    days_since_mens(df, df_dates)

In [80]:
dfs[0].columns

Index(['id', 't', 'cgm', 'basal', 'bolus', 'meals', 'pump', 'clc', 'date',
       'time', 'is mens', 'is ov', 'days since', 'phase'],
      dtype='object')

In [28]:
def uva_cleaning(df):
    '''returns df cleaned with basal delivery/cgm change from cleaned/compiled input'''
    
    # CGM
    # generating rate of change, adding to col
    rate_list = np.zeros(len(df))

    for i in range(len(df)):
        curr_bg = (df.iloc[i]['cgm'])
        last_bg = (df.iloc[i-1]['cgm'])
        rate_list[i] = (curr_bg - last_bg) / 5

    rate_list[0] = np.nan 
    cgmdf = df[['t', 'cgm']].copy()
    cgmdf['cgm change'] = rate_list

    # BASAL
    # starts from rate / hour, converts to rate / 5 mins 
    
    # may need to redo - nan will be same rate, 0 will be a true 0 
    cond_basaldf = df[df['basal'].notnull()]
    cond_basaldf = cond_basaldf[['t', 'basal']]

    basal_cols = ['t', 'basal']
    basal_data = []
    
    for i in range(len(cond_basaldf)):
        # for each basal rate: get duration, delivered, delivery type, 
        # divide up into u delivered per timestamp
        start_time = cond_basaldf.iloc[i]['t']
        
        if i == len(cond_basaldf) - 1:
            end_time = df.iloc[-1]['t']
        else:
            end_time = cond_basaldf.iloc[i+1]['t']
        duration = (end_time - start_time).total_seconds() / 60
        intervals = int(duration // 5)

        units_5min = cond_basaldf.iloc[i]['basal'] / 12
        
        for j in range(intervals):
            # making all timestamps have basal delivered
            new_time = start_time + (j * (pd.Timedelta(minutes=5)))
            row = [new_time, units_5min]
            basal_data.append(row)
    
    if basal_data == []: # if empty, copy over np.nan from orignal df
        basal_data = df[['t', 'basal']]
        
    basaldf = pd.DataFrame(basal_data)
    basaldf.columns = basal_cols
    
    # BOLUS - no expansion needed for this data, just need to have to combine
    bolusdf = df[['t', 'bolus']]
            
    fooddf = df[['t', 'meals']]
    
    mensdf = df[['t', 'date', 'time', 'is mens', 'is ov', 'days since', 'phase']]
    
    # combining
    insulindf = pd.merge(basaldf, bolusdf, how ='outer', on = 't')
    insulindf = insulindf.sort_values(by=['t'], ascending=True)
    intakedf = pd.merge(insulindf, fooddf, how ='outer', on = 't')
    mens_intakedf = pd.merge(intakedf, mensdf, how ='outer', on = 't')
    finaldf = pd.merge(cgmdf, mens_intakedf, how = 'outer', on = 't')
    
    return finaldf

In [41]:
cleaned_dfs = []
for i in range(len(dfs)):
    cleaned_dfs.append(uva_cleaning(dfs[i]))

In [81]:
cleaned_dfs[0].columns

Index(['t', 'cgm', 'cgm change', 'basal', 'bolus', 'meals', 'date', 'time',
       'is mens', 'is ov', 'days since', 'phase', 'day_hour'],
      dtype='object')

In [75]:
def get_baseline_basal(df, time_divs):
    '''returns average basal per 5 minutes across different times'''

    #inds1_1 = df[(df['time'] >= 0) & (df['time'] < time_divs[0])].index.values
    #inds1_2 = df[(df['time'] >= time_divs[3]) & (df['time'] < 24)].index.values
    
    time_chunk1 = df[(df['time'] >= time_divs[0]) & (df['time'] < time_divs[1])]
    time_chunk2 = df[(df['time'] >= time_divs[1]) & (df['time'] < time_divs[2])]
    time_chunk3 = df[(df['time'] >= time_divs[2]) & (df['time'] < time_divs[3])]
    time_chunk4 = df[(df['time'] >= time_divs[3])]
    
    avg_basal1 = time_chunk1['basal'].mean()
    avg_basal2 = time_chunk2['basal'].mean()
    avg_basal3 = time_chunk3['basal'].mean()
    avg_basal4 = time_chunk4['basal'].mean()
    
    avg_schedule = [avg_basal1, avg_basal2, avg_basal3, avg_basal4]

    return avg_schedule

In [76]:
time_divs = ['00:00:00', '00:06:00', '00:12:00', '00:18:00']
get_baseline_basal(cleaned_dfs[0], time_divs)

[0.09332380952380952,
 0.08888968253968252,
 0.08853333333333332,
 0.07477430299371744]

In [74]:
cleaned_dfs[0]['basal'].mean()

0.07499992007848844

# base analysis

In [36]:
def bgi(df):
    '''returns lbgi/hbgi and scaled lbgi/hgbi'''
    df = df.copy()
    df['scaled'] = (1.509*(np.log(df['cgm'])**1.084-5.381)).astype(float)
    df['risk_val'] = 10 *(df['scaled'] ** 2)
    lbgi = len(df[df['scaled'] < 1.0]) / len(df)
    lbgi2 = (df[df['scaled'] < 1.0])['risk_val'].sum() / len(df)
    hbgi = len(df[df['scaled'] > 1.0]) / len(df)
    hbgi2 = (df[df['scaled'] > 1.0])['risk_val'].sum() / len(df)
    
    return [lbgi, lbgi2, hbgi, hbgi2]

In [34]:
# reworking
def cgm_stats(df):
    ''' returns TIR, time<70 (tbr), time>180 (tar), time>250 (tvar), mean, stddev, and CV '''
    # time in range stats
    tot_data = len(df[df['cgm'].notnull()])
    if tot_data != 0 :
        tir = len(df[ (df['cgm'] > 70) & 
                 (df['cgm'] < 180)]) / len(df[df['cgm'].notnull()])
        tbr = len(df[ (df['cgm'] < 70)]) / len(df[df['cgm'].notnull()])
        tar = len(df[ (df['cgm'] > 180)]) / len(df[df['cgm'].notnull()])
        tvar = len(df[ (df['cgm'] > 250)]) / len(df[df['cgm'].notnull()])
    else:
        tir, tbr, tar, tvar = np.nan, np.nan, np.nan, np.nan
    
    # other statistics
    mean = df['cgm'].mean()
    stddev = df['cgm'].std()
    cv = (stddev / mean) * 100
    gmi = 3.31 + (0.02392 * mean)
    lbgi, lbgi2, hbgi, hbgi2 = bgi(df)
    
    results = [tir, tbr, tar, tvar, mean, stddev, cv, gmi, lbgi, lbgi2, hbgi, hbgi2]
    
    return results

In [33]:
def carb_insulin_stats(df):
    ''' returns total insulin, total bolus, total basal, total carbs'''
    total_ins = df['bolus'].sum() + df['basal'].sum()
    total_bolus = df['bolus'].sum()
    total_basal = df['basal'].sum()
    total_carb = df['meals'].sum()
    
    results = [total_ins, total_bolus, total_basal, total_carb]
    return results

In [30]:
def stats_df_row(df):
    '''returns combined cgm/insulin/carb stats with a datestamp to append to df'''
    row = []
    ind = len(df)
    row.append(df.iloc[0]['date'])
    row.append(df.iloc[0]['time'])
    row.append(df.iloc[0]['days since'])
    row = row + cgm_stats(df)
    row = row + carb_insulin_stats(df)
    return row

In [54]:
def stats_df(df, time_div):
    '''returns dataframe of standard statistics from cleaned dataframe'''
    # using day or hour divs for now, likely will want different stats for 5 min intervals
    
    stats = []
    # day
    if time_div == 'days':
        data_dates = df['date'].unique()
        
        for day in data_dates:
            datedf = df[df['date']== day]
            stats.append(stats_df_row(datedf))
        
    # hour - FIX THIS 
    elif time_div == 'hours':
        df['day_hour'] = df['t'].dt.strftime('%m/%d/%Y %H')

        data_hours = df['day_hour'].unique()
        for hour in data_hours:
            hourdf = df[df['day_hour'] == hour]
            stats.append(stats_df_row(hourdf))
        
    cols = ['day', 'hour', 'days_since', 'TIR', 'TBR', 'TAR', 'TVAR', 'mean', 'stddev', 'cv', 'GMI',
            'lbgi', 'lbgi2', 'hbgi', 'hbgi2', 'total_ins', 'total_bolus', 'total_basal', 'total_carb']
    statsdf = pd.DataFrame(stats)
    statsdf.columns = cols
    
    return statsdf

In [58]:
cgm_stats_dfs = []

for df in dfs:
    cgm_df = cgm_stats(df)

In [59]:
# list of stats from cleaned dataframes based on day or time divisions
stats_day_dfs = []
for df in cleaned_dfs:
    statsdf = stats_df(df, 'days') 
    # can be 'days' or 'hours'
    statsdf = statsdf[statsdf['mean'].notnull()] # take out days w no bg data 
    statsdf = statsdf[statsdf['total_ins'] != 0.0] # take out days w no insulin data
    stats_dfs.append(statsdf)

In [60]:
# list of stats from cleaned dataframes based on day or time divisions
stats_hour_dfs = []
for df in cleaned_dfs:
    statsdf = stats_df(df, 'hours') 
    # can be 'days' or 'hours'
    statsdf = statsdf[statsdf['mean'].notnull()] # take out days w no bg data 
    statsdf = statsdf[statsdf['total_ins'] != 0.0] # take out days w no insulin data
    stats_dfs.append(statsdf)

# Block transformation brainstorming

### Fine grained
CGM: 1-5 for 55, 70, 180, 250
Insulin: 1 if below standard, 2 if standard, 3 if above standard for both bolus/basal
- for basal: generate an "average" by getting avg delivery for each time - should be able to just use groupby to get this
- dividing to 12/6 hour segments for average basal delivery? 12-6 overnight - can try multiple methods
- for bolus: need an average bolus , 1 - 2 - 3 for within 20% of avg? need to reevaluate for meals probably or just 1/0
- food 1/0

### Coarse grained
CGM: mean CGM value
tir, tbr, tar
bolus
basal
meals - use 0 for none, 1 2 3 for numbers

- look at overnight time period for weirdness - sort into phases and graph in diff colors? 
- make sure to keep track of what works and what doesn't - helpful to keep a document with approach, and what i did and reasoning
- look at chiara papers for cycle phase splits

In [ ]:
def hashfxn_coarse(df):
    '''more fleshed out hash function with some choices based on significant clinical values/goals'''
    # 'TIR', 'TBR', 'TAR', 'TVAR', 'mean', 'stddev', 'cv', 'GMI', 'lbgi', 'hbgi', 
    # 'incl_low', 'incl_high', 'incl_vhigh', 'total_ins', 'bolus_perc', 'basal_perc'
    vals = ['TIR', 'TBR', 'TAR', 'TVAR', 'mean', 'bolus', 'basal', 'carbs']
    
    hash_list = np.zeros(len(df)).astype(str)
    first_var = True
    
    for var in vals:
        # selecting conditions / choices
         if var == 'TIR': # goal is >.70, 3 = .65 to .75
            conditions = [df[var] <= (.55), df[var] <= (.65), df[var] <= (.75), df[var] <= (.85), .85 < df[var]]
            choices = [1, 2, 3, 4, 5] 
        elif var == 'TBR': # goal is <.05, 3 = .04 - .06
            conditions = [df[var] <= (.02), df[var] <= (.04), df[var] <= (.06), df[var] <= .08, .08 < df[var]]
            choices = [1, 2, 3, 4, 5]
        elif var == 'TAR': # goal is < .25, 3 = .2 - .3
            conditions = [df[var] <= (.1), df[var] <= (.2), df[var] <= .3, df[var] <= .4, .4 < df[var]]
            choices = [1, 2, 3, 4, 5]
        elif var == 'TVAR': # goal is < .05, 3 = .04 - .06
            conditions = [df[var] <= (.02), df[var] <= (.04), df[var] <= (.06), df[var] <= .08, .08 < df[var]]
            choices = [1, 2, 3, 4, 5]
        elif var == 'mean': # basing off of euglycemic goals
            conditions = [df[var] <= (55), df[var] <= (70), df[var] <= (180), df[var] <= 250, 250 < df[var]]
            choices = [1, 2, 3, 4, 5]
                
            
        else: # if within -30, -15, +15, +30% of the mean value for the var
            mean = df[var].mean()
            conditions = [df[var] <= (mean*.70), df[var] <= (mean*.85),df[var] <= mean*1.15,
                df[var] <= mean*1.30, mean*1.30 < df[var]]
            choices = [1, 2, 3, 4, 5]

        # adding vals to list   
        arr = np.select(conditions, choices, default=0)
    
        if first_var == True:
            for i in range(len(hash_list)):
                hash_list[i] = arr[i].astype(str)
            first_var = False
        else:
            for i in range(len(hash_list)):
                hash_list[i] = hash_list[i].astype(str) + arr[i].astype(str)

    df['hash'] = hash_list

In [384]:
stats1.columns

Index(['day', 'hour', 'days_since', 'TIR', 'TBR', 'TAR', 'TVAR', 'mean',
       'stddev', 'cv', 'GMI', 'lbgi', 'lbgi2', 'hbgi', 'hbgi2', 'total_ins',
       'total_bolus', 'total_basal', 'total_carb'],
      dtype='object')

In [385]:
stats1

,day,hour,days_since,TIR,TBR,TAR,TVAR,mean,stddev,cv,GMI,lbgi,lbgi2,hbgi,hbgi2,total_ins,total_bolus,total_basal,total_carb
0,24-Sep-2021,00:00:00,NaN,0.000000,0.0,1.000000,0.714286,253.222222,25.965513,10.254042,9.367076,0.000000,0.000000,0.218750,5.089133,89.45,84.05,5.40,156
1,25-Sep-2021,00:00:00,NaN,0.284722,0.0,0.694444,0.225694,212.045139,40.332348,19.020643,8.382120,0.402778,2.555774,0.597222,12.033252,85.20,82.50,2.70,168
2,26-Sep-2021,00:00:00,NaN,0.000000,0.0,1.000000,0.468750,257.472222,44.977493,17.468872,9.468736,0.020833,0.189736,0.979167,24.098376,73.70,68.30,5.40,73
3,27-Sep-2021,00:00:00,NaN,0.395833,0.0,0.593750,0.097222,193.687500,39.656534,20.474493,7.943005,0.565972,3.222399,0.434028,7.817426,44.70,39.30,5.40,58
4,28-Sep-2021,00:00:00,NaN,0.482639,0.0,0.506944,0.034722,176.763889,46.334639,26.212729,7.538192,0.586806,2.265841,0.413194,6.563632,56.85,54.15,2.70,139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,18-Dec-2021,00:00:00,11.0,0.270833,0.0,0.704861,0.409722,237.041667,66.968491,28.251780,8.980037,0.409722,2.875251,0.590278,17.309949,44.10,38.60,5.50,88
86,19-Dec-2021,00:00:00,12.0,0.024306,0.0,0.972222,0.531250,258.809028,61.840260,23.894166,9.500712,0.090278,0.756593,0.909722,24.008879,59.10,56.35,2.75,85
87,20-Dec-2021,00:00:00,13.0,0.062500,0.0,0.934028,0.333333,230.357639,45.950138,19.947304,8.820155,0.170139,1.101465,0.829861,17.428179,60.55,55.05,5.50,132
88,21-Dec-2021,00:00:00,14.0,0.451389,0.0,0.538194,0.194444,190.031250,60.871222,32.032217,7.855548,0.562500,2.113908,0.437500,9.552921,56.65,53.90,2.75,96
